<a href="https://colab.research.google.com/github/michaeltmk/IBM-Data-Science-Capstone-Project/blob/master/IBM_Data_Science_Specialization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Week 1: Hello World!

This is a capstone project for IBM Data Science Specialization.


In [1]:
import numpy as np
import pandas as pd
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


#Week 3:Segmenting and Clustering Neighborhoods in Toronto


##1. Prepare DataFrame

In [2]:
import requests
import lxml.html as lh
from io import StringIO
!pip install cssselect

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr = doc.cssselect('table.wikitable')

Now, restruceure the text into a right csv format.

In [0]:
str_ = tr[0].text_content().replace('\n',',').replace(',,,','\n').replace(',,','\n')

Form a df and drop Not assigned Borough.

In [4]:
csv = StringIO(str_)
df=pd.read_csv(csv)
df = df[df['Borough']!='Not assigned']
no_neighbour_idxs = df[df['Neighbourhood'] == 'Not assigned'].index
df.loc[no_neighbour_idxs,'Neighbourhood'] = df.loc[no_neighbour_idxs, 'Borough']
df.head(11)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


Merge Neighbourhood in same Postcode.

In [5]:
df_postcode = df['Postcode'].unique()
for postcode in df_postcode:
  Neighbourhoods = df['Neighbourhood'][df['Postcode'] == postcode]
  if len(Neighbourhoods) > 1:
    index_list = Neighbourhoods.index
    df.loc[index_list[0],'Neighbourhood'] = ','.join(Neighbourhoods.to_numpy().squeeze())
    df.drop(labels = index_list[1:], axis = 0, inplace = True)
df.reset_index(drop = True).head(11)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


##2. Get the latitude and the longitude coordinates of each neighborhood.

Get latitude,longitude from geocoder, however this package is very unreliable.

In [0]:
#!pip install geocoder
#import geocoder

def get_coor(postcode,borough):
  lat_lng_coords = None
  while(lat_lng_coords is None):
    g = geocoder.google(f'{postcode},{borough}')
    lat_lng_coords = g.latlng
  latitude,longitude = lat_lng_coords
  return latitude,longitude

#df['latitude'] , df['longitude'] = df.apply(lambda x: get_coor(x.Postcode, x.Borough), axis = 1)
#df.head(11)

Hence, we use data in http://cocl.us/Geospatial_data

In [7]:
csv = lh.fromstring(requests.get('http://cocl.us/Geospatial_data').content)
coor_df = pd.read_csv(StringIO(csv.text_content()))
coor_df.columns = ['Postcode','Latitude','Longitude'] #rename the column for later use
coor_df

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Add the Latitude, Longitude data into the dataframe.

In [8]:
df = df.merge(coor_df, on = 'Postcode')
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.636258,-79.498509


In [9]:
df['Neighbourhood'].astype(np.str) #for the use of below

0                                              Parkwoods
1                                       Victoria Village
2                                           Harbourfront
3                        Lawrence Heights,Lawrence Manor
4                                           Queen's Park
                             ...                        
98           The Kingsway,Montgomery Road,Old Mill North
99                                  Church and Wellesley
100    Business Reply Mail Processing Centre 969 Eastern
101    Humber Bay,King's Mill Park,Kingsway Park Sout...
102    Kingsway Park South West,Mimico NW,The Queensw...
Name: Neighbourhood, Length: 103, dtype: object

##3. Explore and cluster the neighborhoods in Toronto

In [10]:
import folium
from geopy.geocoders import Nominatim
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_ = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_)  
    
map_

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Let's create a function to get venues from each Neighborhoods in Toronto.

In [11]:
#let test on a single Neighborhood first

#api account
VERSION = '20180605'
ACCESS_TOKEN = 'XXXXXXXXXXXXXXXXXXXXXXXX'


LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

neighborhood_latitude, neighborhood_longitude = df.loc[0,['Latitude','Longitude']]
print(df.loc[2,'Postcode'])

url = 'https://api.foursquare.com/v2/venues/explore?&v={}&ll={},{}&radius={}&limit={}&oauth_token={}'.format(
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT,
    ACCESS_TOKEN)
results = requests.get(url).json()
results



M5A


{'meta': {'code': 200, 'requestId': '5e5a6eddbae9a2001bdf5630'},
 'notifications': [{'item': {'unreadCount': 0}, 'type': 'notificationTray'}],
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4e8d9dcdd5fbbbb6b3003c7b-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d163941735',
         'name': 'Park',
         'pluralName': 'Parks',
         'primary': True,
         'shortName': 'Park'}],
       'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'location': {'address': 'Toronto',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 245,
        'formattedAddress': ['Toronto', 'Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          

In [12]:
def getNearbyVenues(postcode, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for code, lat, lng in zip(postcode, latitudes, longitudes):
      while True:
        try:
          #print(code)
              
          # create the API request URL
          url = 'https://api.foursquare.com/v2/venues/explore?&v={}&ll={},{}&radius={}&limit={}&oauth_token={}'.format(
              VERSION, 
              lat, 
              lng, 
              radius, 
              LIMIT,
              ACCESS_TOKEN)
              
          # make the GET request
          results = requests.get(url).json()["response"]['groups'][0]['items']
          
          # return only relevant information for each nearby venue
          venues_list.append([(
              code, 
              lat, 
              lng, 
              v['venue']['name'], 
              v['venue']['location']['lat'], 
              v['venue']['location']['lng'],  
              v['venue']['categories'][0]['name']) for v in results])
          break
        except:
          pass

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Toronto_venues = getNearbyVenues(postcode=df['Postcode'], latitudes=df['Latitude'], longitudes=df['Longitude'])
Toronto_venues.head(11)

,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,Careful & Reliable Painting,43.752622,-79.331957,Construction & Landscaping
2,M3A,43.753259,-79.329656,649 Variety,43.754513,-79.331942,Convenience Store
3,M3A,43.753259,-79.329656,Sun Life,43.754760,-79.332783,Construction & Landscaping
4,M3A,43.753259,-79.329656,GTA Restoration,43.753396,-79.333477,Fireworks Store
5,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
6,M3A,43.753259,-79.329656,Bella Vita Catering & Private Chef Service,43.756651,-79.331524,BBQ Joint
7,M3A,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
8,M3A,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
9,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


We are going to analyze each neighborhood by first restructure the table.
1. One-hot coding for categories of venue
2. Group rows by postcode and by taking the mean of the frequency of occurrence of each category
3. Sort with venues categories for each postcode

In [13]:
#one hot coding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
Toronto_onehot['Postcode'] = Toronto_venues['Postcode']
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]




#group by mean of frequency
Toronto_grouped = Toronto_onehot.groupby('Postcode').mean().reset_index()





#sort with venues categories for each postcode
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

#def function for sorting the most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False) 
    return row_categories_sorted.index.values[0:num_top_venues]

# create a new DF with columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
Toronto_grouped_sorted = pd.DataFrame(columns=columns)
Toronto_grouped_sorted['Postcode'] = Toronto_grouped['Postcode']

for ind in range(Toronto_grouped.shape[0]):
    Toronto_grouped_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

Toronto_grouped_sorted.head(11)

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Home Service,Fast Food Restaurant,Yoga Studio,Fabric Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
1,M1C,Construction & Landscaping,Moving Target,Golf Course,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
2,M1E,Electronics Store,Medical Center,Rental Car Location,Spa,Breakfast Spot,Intersection,Mexican Restaurant,Moving Target,Yoga Studio,Empanada Restaurant
3,M1G,Coffee Shop,Soccer Field,Indian Restaurant,Pharmacy,Mexican Restaurant,Korean Restaurant,Insurance Office,Convenience Store,Greek Restaurant,Empanada Restaurant
4,M1H,Caribbean Restaurant,Bakery,Shipping Store,Hakka Restaurant,Bank,Athletics & Sports,Gym,Lounge,Burger Joint,Gas Station
5,M1J,Women's Store,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Farm
6,M1K,Pharmacy,Discount Store,Train Station,Department Store,Convenience Store,Comfort Food Restaurant,Coffee Shop,Chinese Restaurant,Bus Station,Hobby Shop
7,M1L,Bus Line,Bakery,Metro Station,Intersection,Business Service,Bus Station,Ice Cream Shop,Park,Soccer Field,Farm
8,M1M,Motel,American Restaurant,Skating Rink,Movie Theater,Intersection,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
9,M1N,Skating Rink,General Entertainment,College Stadium,Farm,Café,Yoga Studio,Ethiopian Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant


Cluster Neighborhoods: <font size = 2>(with k-means clustering, k = 5) </font>

In [43]:
from sklearn.cluster import KMeans

kclusters = 5   # set number of clusters
Toronto_grouped_clustering = Toronto_grouped.drop('Postcode', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)   # run k-means clustering

try:
  Toronto_grouped_sorted.insert(0, 'Cluster Labels', kmeans.labels_)   # add clustering labels

  # merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
  df = df.merge(Toronto_grouped_sorted, on='Postcode')
except:    #Cluster Labels already exists if the kmeans clustering already ran once before
  df['Cluster Labels'] = kmeans.labels_

df.head(11)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Construction & Landscaping,BBQ Joint,Convenience Store,Food & Drink Shop,Bus Stop,Fireworks Store,Park,Cuban Restaurant,Farm,Empanada Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Hockey Arena,Financial or Legal Service,Pizza Place,Coffee Shop,Intersection,Portuguese Restaurant,Yoga Studio,Ethiopian Restaurant,Drugstore,Dumpling Restaurant
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Café,Pub,Gym / Fitness Center,Breakfast Spot,Mexican Restaurant,Theater,Shoe Store
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Accessories Store,Women's Store,Home Service,Health & Beauty Service,Coffee Shop,Carpet Store,Sporting Goods Shop,Tailor Shop
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Burger Joint,Park,Diner,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Salad Place,Portuguese Restaurant
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,4,Pizza Place,Yoga Studio,Fabric Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0,Home Service,Fast Food Restaurant,Yoga Studio,Fabric Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
7,M3B,North York,Don Mills North,43.745906,-79.352188,0,Construction & Landscaping,Gym / Fitness Center,Basketball Court,Baseball Field,Café,Restaurant,Japanese Restaurant,Caribbean Restaurant,Pharmacy,Event Space
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937,0,Pizza Place,Pet Store,Intersection,Fast Food Restaurant,Spa,Bus Line,Café,Bank,Furniture / Home Store,Athletics & Sports
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Theater,Bookstore,Burger Joint


Finally, let's visualize the resulting clusters

In [44]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Postcode'], df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##4. Analysis
As the result shows, most of the neighbourhoods are similiar to each other.


In [45]:
df.loc[df['Cluster Labels'] == 0, df.columns[[1] + list(range(5, df.shape[1]))]]    #red

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Hockey Arena,Financial or Legal Service,Pizza Place,Coffee Shop,Intersection,Portuguese Restaurant,Yoga Studio,Ethiopian Restaurant,Drugstore,Dumpling Restaurant
2,Downtown Toronto,0,Coffee Shop,Bakery,Park,Café,Pub,Gym / Fitness Center,Breakfast Spot,Mexican Restaurant,Theater,Shoe Store
3,North York,0,Clothing Store,Furniture / Home Store,Accessories Store,Women's Store,Home Service,Health & Beauty Service,Coffee Shop,Carpet Store,Sporting Goods Shop,Tailor Shop
4,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Burger Joint,Park,Diner,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Salad Place,Portuguese Restaurant
6,Scarborough,0,Home Service,Fast Food Restaurant,Yoga Studio,Fabric Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
...,...,...,...,...,...,...,...,...,...,...,...,...
96,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Steakhouse,American Restaurant,Asian Restaurant,Gastropub,Hotel,Gym,Seafood Restaurant
98,Downtown Toronto,0,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,Burger Joint,Yoga Studio,Gym,Restaurant,Café,Men's Store
99,East Toronto,0,Fast Food Restaurant,Gym / Fitness Center,Light Rail Station,Yoga Studio,Garden Center,Brewery,Skate Park,Smoke Shop,Burrito Place,Spa
100,Etobicoke,0,Construction & Landscaping,Park,Home Service,Business Service,Locksmith,Fabric Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant


In [46]:
df.loc[df['Cluster Labels'] == 1, df.columns[[1] + list(range(5, df.shape[1]))]]     #purple

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,East York,1,Skating Rink,ATM,Asian Restaurant,Cosmetics Shop,Pharmacy,Curling Ice,Dance Studio,Bus Stop,Salon / Barbershop,Construction & Landscaping
19,East Toronto,1,Health Food Store,Neighborhood,Park,Other Great Outdoors,Home Service,Pizza Place,Asian Restaurant,Coffee Shop,Trail,Pub
39,North York,1,Japanese Restaurant,Spa,Chinese Restaurant,Bank,Café,Yoga Studio,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
49,North York,1,Construction & Landscaping,Park,Bakery,Massage Studio,Film Studio,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm
73,Central Toronto,1,Clothing Store,Cosmetics Shop,Spa,Sporting Goods Shop,Salon / Barbershop,Rental Car Location,Coffee Shop,Shoe Store,Seafood Restaurant,Restaurant
97,Etobicoke,1,Yoga Studio,Smoke Shop,Park,River,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


In [47]:
df.loc[df['Cluster Labels'] == 2, df.columns[[1] + list(range(5, df.shape[1]))]]      #blue

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Construction & Landscaping,BBQ Joint,Convenience Store,Food & Drink Shop,Bus Stop,Fireworks Store,Park,Cuban Restaurant,Farm,Empanada Restaurant
20,Downtown Toronto,2,Coffee Shop,Cocktail Bar,Pub,Café,Cheese Shop,Seafood Restaurant,French Restaurant,Farmers Market,Restaurant,Italian Restaurant


In [48]:
df.loc[df['Cluster Labels'] == 3, df.columns[[1] + list(range(5, df.shape[1]))]]     #pale blue

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
92,Downtown Toronto,3,Coffee Shop,Seafood Restaurant,Restaurant,Café,Cocktail Bar,Hotel,Pub,Beer Bar,Italian Restaurant,Japanese Restaurant


In [49]:
df.loc[df['Cluster Labels'] == 4, df.columns[[1] + list(range(5, df.shape[1]))]]     #orange

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,4,Pizza Place,Yoga Studio,Fabric Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
